# Unscented Kalman Filter

The unscented kalman filter is an alternative approach to the extended kalman filter. Instead of linearizing the state transition model and measurement model to compute the probability distribution, the unscented kalman filter computes the posterior distribution using sigma points.

## Motion Models

In the extended Kalman filter module we used one of the most basic motion models where we assumed that the objects were moving at constant velocity. In This section we will explore more advance motion models.

- Constant Turn Rate and Velocity Magnitude (CTRV)
- Constant Turn Rate and Acceleration (CTA)
- Constant Steering Angle and Velocity (CSAV)
- Constant Curvature and Acceleration (CCA)

All of the models can be used with a extended kalman filter or unscented kalman filter.

**Constant Turn Rate and Velocity Model**

This model uses velocity magnitude, yaw angle and yaw rate instead of the cartesian velocity. 

<img src='images/crtv_model.png'/>

The state vector is now represented as

$
x=
\begin{bmatrix}
p_x \\
p_y \\
v \\
\psi \\
\dot{\psi} \\
\end{bmatrix}
$

Now lets derive the state transition function that maps $x$ to $\dot{x}$.

$
\dot{x}=
\begin{bmatrix}
\dot{p}_x \\
\dot{p}_y \\
\dot{v} \\
\dot{\psi} \\
\ddot{\psi} \\
\end{bmatrix}
=
\begin{pmatrix}
v\cdot\cos(\psi) \\
v\cdot\sin(\psi) \\
0 \\
\dot{\psi} \\
0 \\
\end{pmatrix}
$

Since we are assuming constant turn rate this means that the lateral acceleration and radial accelerations are zero. To find the state transition matrix we now need to integrate $\dot{x}$ from $t_k$ to $t_{k+1}$.

$
x_{t_k + 1} = x_{t_k} + \int_{t_k}^{t_{k+1}} \dot{x}
=x_{t_k} + 
\begin{bmatrix}
\frac{v_k}{\dot{\psi}_k}(sin(\psi_k+\dot{\psi}_k \Delta t) - sin(\psi_k)) \\
\frac{v_k}{\dot{\psi}_k}(-cos(\psi_k+\dot{\psi}_k \Delta t) + cos(\psi_k)) \\
0 \\
\dot{\psi}_k \Delta t \\
0 \\
\end{bmatrix}
$

If the yaw rate is zero then the equation becomes

$
x_{t_k + 1} =
x_{t_k} + 
\begin{bmatrix}
v_k cos(\psi_k) \Delta t \\
v_k sin(\psi_k) \Delta t \\
0 \\
0 \\
0 \\
\end{bmatrix}
$

The state transition is composed of a deterministic and stochastic components. In this case the model that we are using is the CRTV. We can compue the state transition function by finding $\dot{x}$ as shown above and integrating from $t_k$ to $t_{k+1}$. The resulting equation is the deterministic component for predicting state $x_{k+1}$.

The part we haven't included yet is the stochastic component. In this model we are assuming a constant turn rate which means we are assuming the longitudinal acceleration and yaw acceleration are zero. In reality this is not the case so we are going to model these two parameters random normal variables with mean, 0. Using these variables we can derive the process covariance matrix.

$
\begin{align}
\nu_{a,k} = \mathcal{N}(0, \sigma^2_{a}) \\
\nu_{\ddot{\psi},l} = \mathcal{N}(0, \sigma^2_{\ddot{\psi}})
\end{align}
$

Lets take a look at how the longitudinal acceleration and yaw acceleration affect the existing model. So the existing model is the following and we want to solve for a, b, c, d, and e using $\eta_{a,k}$ and $\eta_{\ddot{\psi},k}$.

$
x_{t_k + 1} =
x_{t_k} + 
\begin{bmatrix}
\frac{v_k}{\dot{\psi}_k}(sin(\psi_k+\dot{\psi}_k \Delta t) - sin(\psi_k)) \\
\frac{v_k}{\dot{\psi}_k}(-cos(\psi_k+\dot{\psi}_k \Delta t) + cos(\psi_k)) \\
0 \\
\dot{\psi}_k \Delta t \\
0 \\
\end{bmatrix}
+
\begin{bmatrix}
a \\
b \\
c \\
d \\
e \\
\end{bmatrix}
$

$
\begin{align}
a &= \frac{\Delta t^2}{2} cos(\psi_k)\nu_{a,k} \\
b &= \frac{\Delta t^2}{2} cos(\psi_k)\nu_{a,k} \\
c &= \Delta t \nu_{a,k} \\
d &= \frac{\Delta t^2}{2}\nu_{\ddot{\psi},k} \\
e &= \Delta t \nu_{\ddot{\psi},k} \\
\end{align}
$

We can now compute the process covariance, Q using the formula below

$
Q = [\nu_k \cdot \nu_k^T]
$

## Unscented Kalman Filter

The unscented kalman filter handles computing the posterior distribution when the process model or measurement model are nonlinear in a different way than the extended kalman filter. The main problem that is being addressed is the gaussian is not preserved when passed through a nonlinear function. The objective of the unscented kalman filter is to provide the best estimate of a gaussian that represents the true posterior distribution.

The way that it does this is by passing representative points of the original gaussian distribution through the nonlinear function and then computing the covariance of those points. These points are called sigma points. 

**UKF Flow**
- Prediction
    * Generate sigma points
    * Predict sigma points
    * Predict Mean and Covariance
- Update
    * Predict Measurement
    * Update state
    
<img src='images/generating_sigma_points.png'/>

To compute the sigma points the following formula is used

$
X_{k|K} = 
\begin{bmatrix}
x_{k|k} & x_{k|k} + \sqrt{(\lambda + n_x)P_{k|k}} & x_{k|k} - \sqrt{(\lambda + n_x)P_{k|k}}
\end{bmatrix}
$

The $\lambda$ term is a design parameter and adjust the spread of the sigma points. Imagine each column of P and lambda is a scalar multiplier applied to that. A common choice is to set 

$
\lambda = 3 - n_x
$

Below is an implementation in python of a sigma point generator. This can be easily transfered to C++ in the project. I will be using the same parameters that were presented in the module for the state vector and belief covariance.

In [50]:
import numpy as np

# state vector
x = np.array([5.7441, 1.3800, 2.2049, 0.5015, 0.3528])
x = x[np.newaxis].T

# state vector length
nx = x.shape[0]

# lambda
l = 3 - nx

print("x=\n{}".format(x))

P = np.array([[0.0043,   -0.0013,    0.0030,   -0.0022,   -0.0020,],
              [-0.0013,    0.0077,    0.0011,    0.0071,    0.0060,],
              [0.0030,    0.0011,    0.0054,    0.0007,    0.0008,],
              [-0.0022,    0.0071,    0.0007,    0.0098,    0.0100,],
              [-0.0020,    0.0060,    0.0008,    0.0100,    0.0123]])

print("P=\n{}".format(P))

# sqrt of P
A = np.linalg.cholesky(P)

# sigma points
X = np.zeros((5,11), dtype=np.double)
X = np.asmatrix(X)

# compute sigma point array
X[:,0] = x
X[:,1:nx+1] = x + np.sqrt(l + nx)*A
X[:,-nx:] = x - np.sqrt(1 + nx)*A

print("X=\n{}".format(X))

x=
[[ 5.7441]
 [ 1.38  ]
 [ 2.2049]
 [ 0.5015]
 [ 0.3528]]
P=
[[ 0.0043 -0.0013  0.003  -0.0022 -0.002 ]
 [-0.0013  0.0077  0.0011  0.0071  0.006 ]
 [ 0.003   0.0011  0.0054  0.0007  0.0008]
 [-0.0022  0.0071  0.0007  0.0098  0.01  ]
 [-0.002   0.006   0.0008  0.01    0.0123]]
X=
[[ 5.7441      5.85767817  5.7441      5.7441      5.7441      5.7441
   5.58347622  5.7441      5.7441      5.7441      5.7441    ]
 [ 1.38        1.34566241  1.52805719  1.38        1.38        1.38
   1.42856068  1.17061552  1.38        1.38        1.38      ]
 [ 2.2049      2.28414058  2.24556625  2.29582407  2.2049      2.2049
   2.09283689  2.14738924  2.07631395  2.2049      2.2049    ]
 [ 0.5015      0.44339024  0.63188645  0.51692302  0.59522705  0.5015
   0.58367961  0.31710571  0.47968856  0.36894994  0.5015    ]
 [ 0.3528      0.29997295  0.46212294  0.37633934  0.48417035  0.41872062
   0.42750874  0.19819402  0.31951034  0.16701427  0.25957416]]


There is one thing missing now. We need to incorporate the the process model noises in our belief, $\nu_a$ and $\nu_{\dot{\psi}}$. To do this we are going to augment the state vector and covariance matrix to include the process model noise parameters.

$
x_{a}=
\begin{bmatrix}
p_x \\
p_y \\
v \\
\psi \\
\dot{\psi} \\
\nu_a \\
\nu_{\ddot{\psi}} \\
\end{bmatrix}
$

and 

$
P_{a} =
\begin{bmatrix}
P & 0 \\
0 & Q \\
\end{bmatrix}
$

where Q is

$ 
Q=
\begin{bmatrix}
\sigma^2_a & 0 \\
0 & \sigma^2_{\ddot{\psi}} \\
\end{bmatrix}
$

Now that we have compute the sigma points at k, we need to predict where the sigma points will be in k+1 by using the prediction equation with each sigma point as input. For reference the state prediction equations are the following

$
x_{t_k + 1} = x_{t_k} + \int_{t_k}^{t_{k+1}} \dot{x}
=x_{t_k} + 
\begin{bmatrix}
\frac{v_k}{\dot{\psi}_k}(sin(\psi_k+\dot{\psi}_k \Delta t) - sin(\psi_k)) \\
\frac{v_k}{\dot{\psi}_k}(-cos(\psi_k+\dot{\psi}_k \Delta t) + cos(\psi_k)) \\
0 \\
\dot{\psi}_k \Delta t \\
0 \\
\end{bmatrix}
$

If the yaw rate is zero then the equation becomes

$
x_{t_k + 1} =
x_{t_k} + 
\begin{bmatrix}
v_k cos(\psi_k) \Delta t \\
v_k sin(\psi_k) \Delta t \\
0 \\
0 \\
0 \\
\end{bmatrix}
$

After computing the predicted sigma points by passing the points through the state transition function we need to compute the updated belief covariance. To do this we use the following equations

<img src="images/predicted_covariance.png"/>

This completes the prediction update step for the unscented kalman filter. Now we must perform the measurement update. The process is very similar. 

To do the measurement update we are going to use the predicted sigma points generated by the prediction step. These will be passed through the measurement function and the measurement covariance will be computed using the transformed points. In this case the sensor noise is purely additive so we don't need to worry about the sensor covariance when doing this step.

<img src="images/measurement_covariance.png"/>

Now that we have the predicted measurement and measurement covariance all we need to is compute the new state and covariance using the standard kalman filter equations with a minor update. The kalman gain is computed using a cross corelation between the measurement and state covariance.

<img src="images/measurement_update.png"/>